In [61]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

from sklearn.externals.joblib import Parallel, delayed

import pandas as pd
pd.set_option("display.max_columns",101)
pd.options.display.max_rows = 999
import numpy as np
import scipy as sp

import datetime
from collections import Counter
from sklearn.externals.joblib import Parallel, delayed
from IPython.display import display
from scipy import stats

from multiprocessing import Pool
from contextlib import closing

import copy
from itertools import tee
izip = zip
import pickle

from datetime import timedelta

import bayesFisher

import matplotlib.pyplot as plt
%matplotlib inline

import math
import pyjags

In [62]:
def AgeType(x):
    x = int(x)
    if(pd.isnull(x)):
        return(-1)
    if(x<=18):
        return 18
    if(x<=24):
        return 24
    if(x<=53):
        return 53
    if(x<=77):
        return 77
    if(x<=89):
        return 89
    if(x>90):
        return 90
    return 0

def DiagCodeType(x):
    if(pd.isnull(x)):
        return(0)
    if(not str(x).isnumeric()):
        return 1
    x = int(str(x[0:3]))
    if((x>=0)&(x<=139)):
        return 2
    if((x>=140)&(x<=239)):
        return 3
    if((x>=240)&(x<=279)):
        return 4
    if((x>=280)&(x<=289)):
        return 5
    if((x>=290)&(x<=319)):
        return 6
    if((x>=320)&(x<=389)):
        return 7
    if((x>=390)&(x<=459)):
        return 8
    if((x>=460)&(x<=519)):
        return 9
    if((x>=520)&(x<=579)):
        return 10
    if((x>=580)&(x<=629)):
        return 11
    if((x>=630)&(x<=679)):
        return 12
    if((x>=680)&(x<=709)):
        return 13
    if((x>=710)&(x<=739)):
        return 14
    if((x>=740)&(x<=759)):
        return 15
    if((x>=760)&(x<=779)):
        return 16
    if((x>=780)&(x<=799)):
        return 17
    if((x>=800)&(x<=999)):
        return 18
    return 0

def DiagCodeAbbv(x):
    if(pd.isnull(x)):
        return(0)
    if(not str(x).isnumeric()):
        return 1
    x = int(str(x[0:3]))
    return x

In [63]:
def BuildFishersTest(table_label,col,week_vectors,weekend_vectors,data_label):
    week_values = week_vectors[col].values
    weekend_values = weekend_vectors[col].values
    a = weekend_vectors[col].value_counts().get(1,0)
    b = week_vectors[col].value_counts().get(1,0)
    c = weekend_vectors[col].value_counts().get(0,0)
    d = week_vectors[col].value_counts().get(0,0)
    try:
        col_label = data_label[data_label['ITEMID']==col]
        if(col_label.shape[0]==0):
            col_label = str(col)
        else:
            col_label = col_label['LABEL'].iloc[0]
    except:
        col_label = str(col)
    try:
        t = bayesFisher.test(a,b,c,d,prob_diff=0)
        #print(t)
    except:
        return (table_label,col_label,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan)

    return(table_label,col_label,np.mean(week_values),np.mean(weekend_values),t,a,b)

def BuildResultsVector(table_label,a,b,data_label,col):
    try:
        col_label = data_label[data_label['ITEMID']==int(col)]
        #col_label = pd.DataFrame()
        if(col_label.shape[0]==0):
            col_label = str(col)
        else:
            col_label = col_label['LABEL'].iloc[0]
    except:
        col_label = str(col)
    try:
        t = HDIofMCMC(BESTmcmc(a,b,numSavedSteps=10000))
    except:
        return (table_label,col_label,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan)

    return(table_label,col_label,np.mean(a),np.mean(b),t[0],t[1],len(a),len(b))
    

def TimeUntilFirstColumn(table_label,col,table,time_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TimeUntilFirstAction(x['hadm_id'],x['ICU_Time'],data_week,time_col,specific_value=(item_col,col)),axis=1).values
    a = [x for x in a if pd.notnull(x)]

    b = expired_pat_weekend.apply(lambda x: TimeUntilFirstAction(x['hadm_id'],x['ICU_Time'],data_weekend,time_col,specific_value=(item_col,col)),axis=1).values
    b = [x for x in b if pd.notnull(x)]
    return(BuildResultsVector(table_label,a,b,data_label,col))

def TimeBetweenColumn(table_label,col,table,time_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TimeBetweenItems(x['hadm_id'],x['ICU_Time'],data_week,time_col,specific_value=(item_col,col)),axis=1).values
    a = [x for x in a if pd.notnull(x)]

    b = expired_pat_weekend.apply(lambda x: TimeBetweenItems(x['hadm_id'],x['ICU_Time'],data_weekend,time_col,specific_value=(item_col,col)),axis=1).values
    b = [x for x in b if pd.notnull(x)]

    return(BuildResultsVector(table_label,a,b,data_label,col))

def TotalItemColumn(table_label,col,table,time_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TotalItem(x['hadm_id'],x['ICU_Time'],data_week,time_col,(item_col,col)),axis=1).values
    b = expired_pat_weekend.apply(lambda x: TotalItem(x['hadm_id'],x['ICU_Time'],data_weekend,time_col,(item_col,col)),axis=1).values
       
    uno = BuildResultsVector(table_label,list(a),list(b),data_label,col)

    a = [x for x in a if x > 0]
    b = [x for x in b if x > 0]
    
    dos = BuildResultsVector(table_label,list(a),list(b),data_label,col)
    
    return [uno,dos]

def TotalAmountColumn(table_label,col,table,amount_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TotalAmountItem(x['hadm_id'],x['ICU_Time'],data_week,amount_col,(item_col,col)),axis=1)
    b = expired_pat_weekend.apply(lambda x: TotalAmountItem(x['hadm_id'],x['ICU_Time'],data_weekend,amount_col,(item_col,col)),axis=1)
    
    longColumn = []
    longColumn2 = []
    
    for i in [0,1,2,3]:
        temp_a = [x for x in list(a[i].values) if pd.notnull(x)]
        temp_b = [x for x in list(b[i].values) if pd.notnull(x)]
        temp_a_2 = [x for x in list(temp_a) if x > 0]
        temp_b_2 = [x for x in list(temp_b) if x > 0]
        
        longColumn.extend(BuildResultsVector(table_label,temp_a,temp_b,data_label,col))
        longColumn2.extend(BuildResultsVector(table_label,temp_a_2,temp_b_2,data_label,col))
        
    return [longColumn,longColumn2]

def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return izip(a, b)

def TimeBetweenItems(hadm_id,icu_time,table,time_column,specific_value=None):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset.dropna(subset=[time_column],inplace=True)
    if(specific_value!=None):
        subset = subset[subset[specific_value[0]]==specific_value[1]]
    time_diff_list = []
    for (i1, row1), (i2, row2) in pairwise(subset.sort_values(time_column).iterrows()):
        time_diff = float((row2[time_column]-row1[time_column]).total_seconds())/60/60
        if(time_diff>0):
            time_diff_list.append(time_diff)
    if(time_diff_list==[]):
        return np.nan
    return(np.mean(time_diff_list))

def TimeUntilFirstAction(hadm_id,icu_time,table,time_column,specific_value=None):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset.dropna(subset=[time_column],inplace=True)
    if(specific_value!=None):
        subset = subset[subset[specific_value[0]]==specific_value[1]]
    if(subset.shape[0]==0):
        return np.nan
    first_item = subset.sort_values(time_column).head(1)[time_column].iloc[0]
    time_diff = first_item-icu_time
    return(float(time_diff.total_seconds())/60/60)

def TotalItem(hadm_id,icu_time,table,time_column,specific_value):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset = subset[subset[specific_value[0]]==specific_value[1]]
    return(subset.shape[0])

def TotalAmountItem(hadm_id,icu_time,table,amount_col,specific_value):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset = subset[subset[specific_value[0]]==specific_value[1]]
    if(subset.shape[0]==0):
        return pd.Series([np.nan,np.nan,np.nan,np.nan])
    temp = subset[amount_col].values
    temp = [float(x) for x in temp if isinstance(x, (int, int, float, complex))]
    if(len(temp)==0):
        return pd.Series([np.nan,np.nan,np.nan,np.nan])
    return pd.Series([np.nanmean(temp), np.nanmax(temp), np.nanmin(temp), np.nanstd(temp)])

In [64]:
def ReturnNHours(hadm_id,icu_time,table,column,time_window):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    return(subset[(pd.notnull(subset[column])) & (subset[column]>=(icu_time)) & (subset[column]<=(icu_time+datetime.timedelta(hours=time_window)))])

def MakeData(table,column,time_window,expired_pat_weekend,expired_pat_week):
    data_weekend = pd.DataFrame()
    for row in expired_pat_weekend.iterrows():
        data_weekend = pd.concat([data_weekend,ReturnNHours(row[1]['hadm_id'],row[1]['ICU_Time'],table,column,time_window)])
        
    data_week = pd.DataFrame()
    for row in expired_pat_week.iterrows():
        data_week = pd.concat([data_week,ReturnNHours(row[1]['hadm_id'],row[1]['ICU_Time'],table,column,time_window)])
    return((data_weekend,data_week))


def papply(groups, fxn, pmax=5):
    n = min(len(groups), pmax)    
    with closing(Pool(processes=n)) as p:
        result = p.map(fxn, [g for _, g in groups])
    return pd.concat(result)
    pool.terminate()


def INPUTEVENTS_fxn_by_group(df):
    i = df.concept_id.values[0]
    newVal = df[(df.label.notnull())]
    if not newVal.empty:
        df.concept_id = df.concept_id.replace(i,str(newVal.label.unique()[0]))
    return df  

def PROCEDUREEVENTS_fxn_by_group(df):
    i = df.concept_id.values[0]
    newVal = df[(df.ORDERCATEGORYDESCRIPTION.notnull())]
    if not newVal.empty:
        df.concept_id = df.concept_id.replace(i,str(newVal.ORDERCATEGORYDESCRIPTION.unique()[0]))
    return df  

def LABEVENTS_fxn_by_group(df):
    i = df.value_as_concept_id.values[0]
    newVal = df[(df.LABEL.notnull())]
    if not newVal.empty:
        df.value_as_concept_id = df.value_as_concept_id.replace(i,str(newVal.LABEL.unique()[0]))
    return df  

def CHARTEVENTS_fxn_by_group(df):
    i = df.unit_concept_id.values[0]
    newVal = df[(df.LABEL.notnull())]
    if not newVal.empty:
        df.unit_concept_id = df.unit_concept_id.replace(i,str(newVal.LABEL.unique()[0]))
    return df  

def LoadMIMICTables(table_name,time_columns,time_formats):
    table = pd.read_csv('../MIMICIII/'+table_name+'.csv')
    #Generate concept ids
    if(table_name == 'MICROBIOLOGYEVENTS'):
        maps = pd.read_csv('map/microbiology_specimen_to_concept.csv')
        table = table.merge(maps, left_on='SPEC_TYPE_DESC', right_on='label', how='left').drop('label',1)
        table.concept_name = table.concept_name.fillna(table.SPEC_TYPE_DESC)
        table.drop('SPEC_TYPE_DESC',1,inplace=True)
        table.rename(columns={"concept_name": "SPEC_TYPE_DESC"},inplace=True)     
    if(table_name == 'PROCEDUREEVENTS_MV'):
        maps = pd.read_csv('map/procedure_to_concept.csv')
        table = table.merge(maps, left_on='ITEMID', right_on='item_id', how='left').drop('item_id',1)
        table.concept_id = table.concept_id.fillna(table.ITEMID)
        table.drop('ITEMID',1,inplace=True)
        table.rename(columns={"concept_id": "ITEMID"},inplace=True)
    if(table_name == 'INPUTEVENTS_CV'):
        maps = pd.read_csv('map/cv_input_label_to_concept.csv')
        table = table.merge(maps, left_on='ITEMID', right_on='item_id', how='left').drop('item_id',1)
        table.concept_id = table.concept_id.fillna(table.ITEMID)
        table.drop('ITEMID',1,inplace=True)
        table.rename(columns={"concept_id": "ITEMID"},inplace=True)
    if(table_name == 'INPUTEVENTS_MV'):
        maps = pd.read_csv('map/mv_input_label_to_concept.csv')
        table = table.merge(maps, left_on='ITEMID', right_on='item_id', how='left').drop('item_id',1)
        table.concept_id = table.concept_id.fillna(table.ITEMID)
        table.drop('ITEMID',1,inplace=True)
        table.rename(columns={"concept_id": "ITEMID"},inplace=True)
    if(table_name == 'LABEVENTS'):
        maps = pd.read_csv('map/lab_label_to_concept.csv')
        d_lab_items_for_maps = pd.read_csv('../MIMICIII/D_LABITEMS.csv')
        table = table.merge(d_lab_items_for_maps[['ITEMID','LABEL']], on='ITEMID').merge(maps, left_on='LABEL', right_on='label', how='left')
        table.concept_id = table.concept_id.fillna(table.ITEMID)
        table.drop('ITEMID',1,inplace=True)
        table.rename(columns={"concept_id": "ITEMID"},inplace=True)       
    if(table_name == 'CHARTEVENTS'):
        d_items_for_maps = pd.read_csv('../MIMICIII/D_ITEMS.csv')
        table = table.merge(d_items_for_maps[['ITEMID','LABEL']], on='ITEMID', how='left')
        maps = pd.read_csv('map/chart_label_to_concept.csv')
        table = table.merge(maps[['d_label','unit_concept_id']], left_on='LABEL', right_on='d_label', how='left')
        table.unit_concept_id = table.unit_concept_id.replace(8510,8510000000)
        table.unit_concept_id = table.unit_concept_id.replace(8554,8554000000)
        table.unit_concept_id = table.unit_concept_id.fillna(table.ITEMID)
        table.drop('ITEMID',1,inplace=True)
        table.rename(columns={"unit_concept_id": "ITEMID"},inplace=True) 
    if(table_name == 'CPTEVENTS'):
        d_items_for_maps = pd.read_csv('cptmap.csv')
        table = table.merge(d_items_for_maps, left_on='CPT_CD', right_on='code', how='left')
        table.concept = table.concept.fillna(table.CPT_CD)
        table.drop('CPT_CD',1,inplace=True)
        table.rename(columns={"concept": "CPT_CD"},inplace=True) 
                
    #Clean Data
    table = table[table['HADM_ID'].isin(expired_pat_id)]
    for index,item in enumerate(time_columns):
        table[item] = pd.to_datetime(table[item], format=time_formats[index])
    return(table)

def GetPossibleItems(table,column,week,weekend):
    #possible_items = table[column].unique()
    possible_items_wknd = [x[0] for x in Counter(weekend[column]).most_common(100)]
    possible_items_wknd = [x for x in possible_items_wknd if pd.notnull(x)]
    
    possible_items_wk = [x[0] for x in Counter(week[column]).most_common(100)]
    possible_items_wk = [x for x in possible_items_wk if pd.notnull(x)]
    
    return (list(set(possible_items_wk).intersection(set(possible_items_wknd))))

def MakeItemVectors(table,column,data_weekend,data_week):
    vector_mapping = {}
    vector_list = []
    possible_items = GetPossibleItems(table,column,data_week,data_weekend)
    possible_items_set = set(possible_items)
    for index,item in enumerate(possible_items):
        vector_list.append(item)
        vector_mapping[item] = index
    
    data_weekend_vectors = []
    for row in data_weekend['HADM_ID'].unique():
        temp_vector = np.zeros(len(possible_items))
        for row1 in table[table['HADM_ID']==row][column].unique():
            if((pd.isnull(row1)) | (row1 not in possible_items_set) ):
                continue
            temp_vector[vector_mapping[row1]] = 1
        data_weekend_vectors.append(list(temp_vector))
        
    data_week_vectors = []
    for row in data_week['HADM_ID'].unique():
        temp_vector = np.zeros(len(possible_items))
        for row1 in table[table['HADM_ID']==row][column].unique():
            if((pd.isnull(row1)) | (row1 not in possible_items_set) ):
                continue
            temp_vector[vector_mapping[row1]] = 1
        data_week_vectors.append(list(temp_vector))
    return((data_weekend_vectors,data_week_vectors,vector_list))

def get_rxnorm_ingredients(rxcui):
    """
    Return a generator of RxNorm ingredients related to the provided RxNorm concept.
    Each ingredient is a (rxcui, name, umlscui) tuple. This function calls the
    RxNormAPI as documented here:
    http://rxnav.nlm.nih.gov/RxNormAPIs.html#uLink=RxNorm_REST_getAllRelatedInfo
    """
    base_uri = 'http://rxnav.nlm.nih.gov/REST'
    url = '{base_uri}/rxcui/{rxcui}/allrelated'.format(base_uri = base_uri, rxcui = rxcui)
    response = requests.get(url)
    tree = xml.etree.ElementTree.fromstring(response.text)
    xml_ingredients = tree.findall("./allRelatedGroup/conceptGroup[tty='IN']/conceptProperties")
    for xml_ingredient in xml_ingredients:
        assert xml_ingredient.findtext('tty') == 'IN'
        yield tuple(xml_ingredient.findtext(tag) for tag in ['rxcui', 'name', 'umlscui'])

In [65]:
def gammaShRaFromModeSD(mode, sd):
    #if this flags then all values from y1 and y2 are identical
    if(mode <=0):
        #print("mode must be > 0")
        return
    if(sd <=0):
        #print("sd must be > 0")
        return
    rate = ( mode + math.sqrt( mode**2 + 4 * sd**2 ) ) / ( 2 * sd**2 )
    shape = 1 + mode * rate
    return({'shape':shape,'rate':rate})

def gammaShRaFromMeanSD( mean , sd ):
    if(mean <=0):
        print("mean must be > 0")
        return
    if(sd <=0):
        print("sd must be > 0")
        return
    shape = mean**2/sd**2
    rate = mean/sd**2
    return({'shape':shape,'rate':rate})

def BESTmcmc(y1, y2, numSavedSteps=100000, thinSteps=1):
    #MODEL
    modelString = '''
    model {
    for ( i in 1:Ntotal ) {
    y[i] ~ dt( mu[x[i]] , 1/sigma[x[i]]^2 , nu )
    }
    mu[1] ~ dgamma( Sh1 , Ra1 )  # prior for mu[1]
    sigma[1] ~ dgamma( Sh1 , Ra1 )     # prior for sigma[1]
    mu[2] ~ dgamma( Sh2 , Ra2 )  # prior for mu[2]
    sigma[2] ~ dgamma( Sh2 , Ra2 )     # prior for sigma[2]
    nu <- nuMinusOne+1
    nuMinusOne ~ dgamma( ShNu , RaNu ) # prior for nu
    }
    ''' 
    
    #DATA
    y = np.array(y1+y2) #combine data into one vector
    l1 = np.repeat(1, len(y1), axis=0).tolist()
    l2 = np.repeat(2, len(y2), axis=0).tolist()
    x = np.array(l1+l2) #create group membership code
    Ntotal = len(y)
    #specify the data and prior constants in a list, for later shipment to "JAGS"
    dataList = {
        'y': y,
        'x': x,
        'Ntotal': Ntotal,
        # Below are the specifications for the prior constants. These are generic
        # broad priors, but you can replace with informed values if appropriate.  
#         'muM1': np.mean(y),
#         'muSD1': np.std(y,ddof=1)*5,
#         'muM2': np.mean(y),
#         'muSD2': np.std(y,ddof=1)*5,
        'Sh1': gammaShRaFromModeSD(mode=np.std(y,ddof=1), sd=np.std(y,ddof=1)*5)['shape'],
        'Ra1': gammaShRaFromModeSD(mode=np.std(y,ddof=1), sd=np.std(y,ddof=1)*5)['rate'],
        'Sh2': gammaShRaFromModeSD(mode=np.std(y,ddof=1), sd=np.std(y,ddof=1)*5)['shape'],
        'Ra2': gammaShRaFromModeSD(mode=np.std(y,ddof=1), sd=np.std(y,ddof=1)*5)['rate'],
        'ShNu': gammaShRaFromMeanSD(mean=29, sd=29)['shape'],
        'RaNu': gammaShRaFromMeanSD(mean=29, sd=29)['rate']
    }
    
    
    #INIALIZE THE CHAINS
    # Initial values of MCMC chains based on data:
    mu = [np.mean(y1),np.mean(y2)]
    sigma = [np.std(y1,ddof=1),np.std(y2,ddof=1)]
    # Regarding initial values in next line: (1) sigma will tend to be too big if 
    # the data have outliers, and (2) nu starts at 5 as a moderate value. These
    # initial values keep the burn-in period moderate. 
    initsList = {'mu':mu, 'sigma':sigma, 'nuMinusOne':4}
    
    #RUN THE CHAINS
    parameters = ['mu','sigma','nu'] # The parameters to be monitored
    adaptSteps = 500                 # Number of steps to "tune" the samplers
    burnInSteps = 1000
    nChains = 3 
    nIter = math.ceil( ( numSavedSteps * thinSteps ) / nChains )    
    
    # Create, initialize, and adapt the model:
    model = pyjags.Model(code=modelString, data=dataList, init=initsList, chains=nChains, adapt=adaptSteps, threads=40)
    model.update(burnInSteps)
    codaSamples = model.sample(iterations=nIter, vars=parameters, thin=thinSteps)
    
    return codaSamples

def HDIofMCMC(sampleVec, credMass=0.95):
    sampleVec = np.subtract(sampleVec['mu'][0].ravel(), sampleVec['mu'][1].ravel())
    
    sortedPts = np.sort(sampleVec)
    ciIdxInc = math.floor(credMass * len(sortedPts))
    nCIs = len(sortedPts) - ciIdxInc
    ciWidth = np.repeat(0, nCIs, axis=0).tolist()
    for i in range(0,nCIs):
        ciWidth[i] = np.subtract(sortedPts[i + ciIdxInc], sortedPts[i])
    ciWidth = np.array(ciWidth)
    HDImin = sortedPts[np.argmin(ciWidth)]
    HDImax = sortedPts[np.argmin(ciWidth) + ciIdxInc]
    HDIlim = np.array([HDImin, HDImax])
    return(HDIlim)

# LOAD DATA ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [66]:
#Load Data
pat_data = pd.read_pickle('pat_data.pkl')

#Clean Data
pat_data['Primary_Diag_Code'] = pat_data['Primary_Diag'].apply(lambda x: DiagCodeType(x))
pat_data['DiagCodeAbbv'] = pat_data['Primary_Diag'].apply(lambda x: DiagCodeAbbv(x))
pat_data = pat_data[pat_data.Primary_Diag_Code > 0]
pat_data['Age'] = pat_data['Age'].apply(lambda x: AgeType(x))
pat_data = pat_data.replace(to_replace=['UNABLE TO OBTAIN','UNKNOWN/NOT SPECIFIED','PATIENT DECLINED TO ANSWER'], value='UNKNOWN')
pat_data = pat_data.replace(to_replace=['ASIAN - CHINESE','ASIAN - VIETNAMESE','ASIAN - ASIAN INDIAN','ASIAN - FILIPINO','ASIAN - KOREAN','ASIAN - OTHER','ASIAN - CAMBODIAN','ASIAN - JAPANESE','ASIAN - THAI'], value='ASIAN')
pat_data = pat_data.replace(to_replace=['HISPANIC/LATINO - PUERTO RICAN','HISPANIC/LATINO - DOMINICAN','HISPANIC/LATINO - GUATEMALAN','HISPANIC/LATINO - SALVADORAN','HISPANIC/LATINO - CUBAN','HISPANIC/LATINO - MEXICAN','HISPANIC/LATINO - COLOMBIAN','HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)','HISPANIC/LATINO - HONDURAN'], value='HISPANIC OR LATINO')
pat_data = pat_data.replace(to_replace=['BLACK/AFRICAN AMERICAN','BLACK/CAPE VERDEAN','BLACK/HAITIAN','BLACK/AFRICAN'], value='BLACK')
pat_data = pat_data.replace(to_replace=['WHITE - RUSSIAN','WHITE - OTHER EUROPEAN','WHITE - BRAZILIAN','WHITE - EASTERN EUROPEAN'], value='WHITE')
pat_data = pat_data.replace(to_replace=['MULTI RACE ETHNICITY','AMERICAN INDIAN/ALASKA NATIVE','PORTUGUESE','MIDDLE EASTERN','NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER','SOUTH AMERICAN','CARIBBEAN ISLAND','AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE'], value='OTHER')
pat_data = pat_data[pat_data.Admit_Type != 'NEWBORN']
pat_data = pat_data[pat_data.Admit_Unit != 'NICU']
pat_data = pat_data[pat_data.Lag_Time > 0]
pat_data = pat_data.replace(to_replace=['TRANSFER FROM SKILLED NUR','TRANSFER FROM OTHER HEALT','TRSF WITHIN THIS FACILITY','TRANSFER FROM HOSP/EXTRAM'], value='TRANSFER')
pat_data = pat_data[pat_data.Admit_LOC.isin(['EMERGENCY ROOM ADMIT','TRANSFER','CLINIC REFERRAL/PREMATURE','PHYS REFERRAL/NORMAL DELI'])]
pat_data = pat_data.replace(to_replace=['CLINIC REFERRAL/PREMATURE','PHYS REFERRAL/NORMAL DELI'], value='REFERRAL')
pat_data.loc[(pat_data.DOW == 5)|(pat_data.DOW == 6), 'weekend'] = 1
#pat_data.loc[(pat_data.DOW == 6)&(pat_data.TOD == 'Night'), 'weekend'] = 1
pat_data.weekend = pat_data.weekend.fillna(0)
pat_data['LOS_ICU_o'] = pat_data['LOS_ICU']
pat_data['LOS_ICU'] = pd.qcut(pat_data['LOS_ICU'], 4,False)
pat_data['Hospital_LOS'] = pd.qcut(pat_data['Hospital_LOS'], 4,False)
pat_data = pat_data[pat_data['Expire']==1]

#Merging
icu_stay_table = pd.read_csv('../MIMICIII/ICUSTAYS.csv')
icu_stay_table.INTIME = pd.to_datetime(icu_stay_table.INTIME)
pat_data.ICU_Time = pd.to_datetime(pat_data.ICU_Time)
pat_data = pat_data.merge(icu_stay_table[['HADM_ID','ICUSTAY_ID','INTIME']], left_on=['hadm_id','ICU_Time'], right_on=['HADM_ID','INTIME']).drop(['HADM_ID','INTIME'],1)

oasis = pd.read_csv('../views/oasis.csv')
pat_data = pat_data.merge(oasis[['hadm_id','icustay_id','oasis_prob']], left_on=['hadm_id','ICUSTAY_ID'], right_on=['hadm_id','icustay_id'], how='left')
pat_data['oasis_prob'] = pd.qcut(pat_data['oasis_prob'],4,labels=['a','b','c','d']).astype(str)

pat_data = pat_data.drop(['ICUSTAY_ID','icustay_id'],1)

In [67]:
subset = pat_data[(pat_data.DiagCodeAbbv == 38)&(pat_data.Admit_Unit == 'MICU')&(pat_data.Admit_Type == 'EMERGENCY')]

In [68]:
subset.weekend.value_counts()

0.0    318
1.0    140
Name: weekend, dtype: int64

In [8]:
subset.shape

(458, 26)

In [9]:
#Controlled Data Test
expired_pat_weekend = subset[(subset['DOW'].isin([5,6]))]
expired_pat_week = subset[(subset['DOW'].isin([0,1,2,3,4]))]
expired_pat_id = pat_data['hadm_id'].values

In [10]:
#d_items = pd.read_csv('../MIMICIII/D_ITEMS.csv')
#d_lab_items = pd.read_csv('../MIMICIII/D_LABITEMS.csv')
d_items = pd.read_csv('d_items_maps.csv')
d_lab_items = pd.read_csv('d_lab_items_maps.csv')
#Check Datetime

# ANALYSIS. |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [11]:
allResults = pd.DataFrame(columns=['analysis','table','column','week','weekend','U-Stat','P-Val','weekendCount','weekCount'])

In [12]:
#Mapped files
clinical_data = [('MICROBIOLOGYEVENTS','CHARTTIME','SPEC_ITEMID','NA','%Y-%m-%d %H:%M:%S'),('LABEVENTS','CHARTTIME','ITEMID','VALUENUM','%Y-%m-%d %H:%M:%S'),
                 ('PRESCRIPTIONS','STARTDATE','DRUG','DOSE_VAL_RX','%Y-%m-%d %H:%M:%S'),('PROCEDUREEVENTS_MV','STARTTIME','ITEMID','VALUE','%Y-%m-%d %H:%M:%S'),
                 ('CPTEVENTS','CHARTDATE','CPT_CD','NA','%Y-%m-%d %H:%M:%S'),('DATETIMEEVENTS','CHARTTIME','ITEMID','NA','%Y-%m-%d %H:%M:%S'),
                 ('NOTEEVENTS','CHARTDATE','CATEGORY','NA','%Y-%m-%d'),('CHARTEVENTS','CHARTTIME','ITEMID','VALUENUM','%Y-%m-%d %H:%M:%S')]

# clinical_data_tables = []
# for table,time_col,item_col,amount_col,date_label in clinical_data:
#     proc_table = LoadMIMICTables(table,[time_col],[date_label])
#     if(table=='LABEVENTS'):
#         clinical_data_tables.append((table,time_col,item_col,amount_col,proc_table,d_lab_items))
#     else:
#         clinical_data_tables.append((table,time_col,item_col,amount_col,proc_table,d_items))

# input_cv = LoadMIMICTables('INPUTEVENTS_CV',['CHARTTIME'],['%Y-%m-%d %H:%M:%S'])
# input_mv = LoadMIMICTables('INPUTEVENTS_MV',['STARTTIME'],['%Y-%m-%d %H:%M:%S'])
# input_mv = input_mv.rename(index=str, columns={"STARTTIME": "CHARTTIME"})
# input_columns = ['HADM_ID','CHARTTIME','ITEMID','AMOUNT','CGID']
# input_data = pd.concat([input_cv[input_columns].reset_index(drop=True),input_mv[input_columns].reset_index(drop=True)])
# clinical_data_tables.append(('INPUTITEMS','CHARTTIME','ITEMID','AMOUNT',input_data,d_items))
with open('clinical_data_tables.pkl', 'rb') as f:
    clinical_data_tables = pickle.load(f)

time_span = 24

d_items[d_items.LABEL == 'Levophed-k']

input_mv = LoadMIMICTables('INPUTEVENTS_MV',['STARTTIME'],['%Y-%m-%d %H:%M:%S'])
input_cv = LoadMIMICTables('INPUTEVENTS_CV',['CHARTTIME'],['%Y-%m-%d %H:%M:%S'])

In [45]:
#test = clinical_data_tables[1][4]
#test[test.ITEMID == 51003.0].head()

input_cv[input_cv.ITEMID == 30120.0].head()

In [13]:
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
    vector_items = MakeItemVectors(table,item_col,data_weekend,data_week)
    weekend_vectors = pd.DataFrame(vector_items[0],columns=vector_items[2])
    week_vectors = pd.DataFrame(vector_items[1],columns=vector_items[2])

    class_weekend = list([1]*weekend_vectors.shape[0])
    class_week = list([0]*week_vectors.shape[0])
    classes = class_weekend+class_week
    #print(week_vectors.shape)
    #print(weekend_vectors.shape)

    data = []
    for col in vector_items[2]:
        data.append(BuildFishersTest(table_label,col,week_vectors,weekend_vectors,data_label))

    #data = pd.DataFrame(data,columns=['Column','% Week','%Weekend','P-Val','weekendCount','weekCount'])
    data = pd.DataFrame(data,columns=['table','column','week','weekend','P-Val','weekendCount','weekCount'])
    data = data.dropna()
    data = data[(data['P-Val']<0.05)|(data['P-Val']>0.95)]   
    
    #append things to stuff
    data['analysis'] = 'Fishers'
    display(data.head())
    allResults = allResults.append(data)

,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis
26,LABEVENTS,Base Excess,0.947883,0.881481,0.007511,119,291,Fishers
27,LABEVENTS,Calculated Total CO2,0.947883,0.881481,0.007511,119,291,Fishers
34,LABEVENTS,Oxygen Saturation,0.723127,0.637037,0.034933,86,222,Fishers
35,LABEVENTS,pCO2,0.947883,0.881481,0.007511,119,291,Fishers
37,LABEVENTS,pH,0.957655,0.903704,0.014408,122,294,Fishers


/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis
6,PRESCRIPTIONS,Midazolam,0.398577,0.296610,0.027388,35,112,Fishers
17,PRESCRIPTIONS,Midazolam HCl,0.170819,0.245763,0.958616,29,48,Fishers
21,PRESCRIPTIONS,Cisatracurium Besylate,0.088968,0.152542,0.969003,18,25,Fishers
26,PRESCRIPTIONS,Phenylephrine,0.231317,0.144068,0.024999,17,65,Fishers
40,PRESCRIPTIONS,Soln,0.266904,0.177966,0.029614,21,75,Fishers


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis
20,PROCEDUREEVENTS_MV,Multi Lumen,0.738255,0.534483,0.002656,31,110,Fishers
27,PROCEDUREEVENTS_MV,Invasive Ventilation,0.671141,0.500000,0.011720,29,100,Fishers
36,PROCEDUREEVENTS_MV,Paracentesis,0.040268,0.103448,0.959310,6,6,Fishers


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis
1,CPTEVENTS,94003,0.590909,0.428571,0.013499,27,104,Fishers
2,CPTEVENTS,94003,0.306818,0.476190,0.991718,30,54,Fishers


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis
0,DATETIMEEVENTS,Multi Lumen Cap Change,0.408730,0.271845,0.007556,28,103,Fishers
2,DATETIMEEVENTS,Multi Lumen Dressing Change,0.412698,0.281553,0.010233,29,104,Fishers
3,DATETIMEEVENTS,Multi Lumen Insertion Date,0.424603,0.271845,0.003456,28,107,Fishers
4,DATETIMEEVENTS,Multi Lumen Tubing Change,0.396825,0.271845,0.013042,28,100,Fishers
53,DATETIMEEVENTS,Presep Catheter Tubing Change,0.027778,0.067961,0.960963,7,7,Fishers


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis
10,CHARTEVENTS,Tidal Volume (set),0.754717,0.666667,0.026323,92,240,Fishers
78,CHARTEVENTS,PEEP set,0.757862,0.666667,0.022196,92,241,Fishers


,table,column,week,weekend,P-Val,weekendCount,weekCount,analysis
7,INPUTITEMS,Fentanyl (Concentrate),0.218354,0.131387,0.015381,18,69,Fishers
13,INPUTITEMS,Fentanyl,0.208861,0.291971,0.972324,40,66,Fishers
16,INPUTITEMS,Thiamine,0.063291,0.014599,0.012695,2,20,Fishers
25,INPUTITEMS,Propofol,0.079114,0.138686,0.974897,19,25,Fishers
26,INPUTITEMS,Phenylephrine,0.227848,0.145985,0.023697,20,72,Fishers


In [ ]:
#Time version
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
    possible_items = GetPossibleItems(table,item_col,data_week,data_weekend)
    #data = Parallel(n_jobs=-1)(delayed(TimeUntilFirstColumn)(col,table,time_col,item_col,data_label) for col in possible_items)
    data = []
    for col in possible_items:
        data.append(TimeUntilFirstColumn(table_label,col,table,time_col,item_col,data_label))
    
    print("Time Until First Action")
    print(table_label)
    data = pd.DataFrame(data,columns=['table','column','week','weekend','U-Stat','P-Val','weekendCount','weekCount'])
    data = data.dropna()    
    data = data[((data['U-Stat']>0)&(data['P-Val']>0))|((data['U-Stat']<0)&(data['P-Val']<0))]
    data['analysis'] = 'Time Until First Action'
    display(data.head())
    allResults = allResults.append(data)
    print(' ')
    
    #data = Parallel(n_jobs=-1)(delayed(TimeBetweenColumn)(col,table,time_col,item_col,data_label) for col in possible_items)
    data = []
    for col in possible_items:
        data.append(TimeBetweenColumn(table_label,col,table,time_col,item_col,data_label))
    
    print("Time Between")
    print(table_label)
    data = pd.DataFrame(data,columns=['table','column','week','weekend','U-Stat','P-Val','weekendCount','weekCount'])
    data = data.dropna()
    data = data[((data['U-Stat']>0)&(data['P-Val']>0))|((data['U-Stat']<0)&(data['P-Val']<0))]
    data['analysis'] = 'Time Between'
    display(data.head())
    allResults = allResults.append(data)
    print(' ')

/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Time Until First Action
MICROBIOLOGYEVENTS


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)


Time Between
MICROBIOLOGYEVENTS


...

,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis
41,CHARTEVENTS,Urine [Color],4.084034,3.842997,0.001019,0.319468,150,66,Time Between
82,CHARTEVENTS,Oral Cavity,4.331224,4.028379,0.091096,0.539511,137,56,Time Between


 
Time Until First Action
INPUTITEMS


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis
13,INPUTITEMS,Fentanyl,8.084519,5.407415,0.041259,5.537312,52.0,26.0,Time Until First Action


 
Time Between
INPUTITEMS


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


In [ ]:
#Total Items
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables:  
    data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
    possible_items = GetPossibleItems(table,item_col,data_week,data_weekend)

    #data = Parallel(n_jobs=-1,max_nbytes=1e6)(delayed(TotalItemColumn)(col,table,time_col,item_col,data_label) for col in possible_items)
    data1 = []
    data2 = []
    for col in possible_items:
        results = TotalItemColumn(table_label,col,table,time_col,item_col,data_label)
        data1.append(results[0])
        data2.append(results[1])
        
    print("Total Number of Each Item")
    print(table_label)
    data1 = pd.DataFrame(data1,columns=['table','column','week','weekend','U-Stat','P-Val','weekendCount','weekCount']).dropna()
    data1 = data1[((data1['U-Stat']>0)&(data1['P-Val']>0))|((data1['U-Stat']<0)&(data1['P-Val']<0))]
    data1['analysis'] = 'Total Number of Each Item, no nans'
          
    data2 = pd.DataFrame(data1,columns=['table','column','week','weekend','U-Stat','P-Val','weekendCount','weekCount']).dropna()
    data2 = data2[((data2['U-Stat']>0)&(data2['P-Val']>0))|((data2['U-Stat']<0)&(data2['P-Val']<0))]
    data2['analysis'] = 'Total Number of Each Item, no 0s'
    
    display(data1.head())
    display(data2.head())
    allResults = allResults.append(data1)
    allResults = allResults.append(data2)
    print(' ')

/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Total Number of Each Item
MICROBIOLOGYEVENTS


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


 
Total Number of Each Item
LABEVENTS


...

,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


 
Total Number of Each Item
CHARTEVENTS


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis
27,CHARTEVENTS,Non Invasive Blood Pressure systolic,114.18239,101.014286,1.101279,19.117164,318.0,140.0,"Total Number of Each Item, no nans"


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis
27,CHARTEVENTS,Non Invasive Blood Pressure systolic,114.18239,101.014286,1.101279,19.117164,318.0,140.0,"Total Number of Each Item, no 0s"


In [ ]:
#Make Amount items
amount_data = set(['LABEVENTS','PRESCRIPTIONS_mapped','PROCEDUREEVENTS_MV','CHARTEVENTS','INPUTITEMS'])
#Total Item Amount Items
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables:
    print(table_label)
    if(table_label in amount_data): 
        data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
        possible_items = GetPossibleItems(table,item_col,data_week,data_weekend)
        
        #data = Parallel(n_jobs=-1,max_nbytes=1e6)(delayed(TotalAmountColumn)(col,table,amount_col,item_col,data_label) for col in possible_items)
        data1 = []
        data2 = []
        for col in possible_items:
            results = TotalAmountColumn(table_label,col,table,amount_col,item_col,data_label)
            data1.append(results[0])
            data2.append(results[1])

            
        colss = ['table_mean','Column_mean','CountWeek_mean','CountWeekend_mean','U-Stat_mean','P-Val_mean','weekendCount_mean','weekCount_mean',
                 'table_max','Column_max','CountWeek_max','CountWeekend_max','U-Stat_max','P-Val_max','weekendCount_max','weekCount_max',
                 'table_min','Column_min','CountWeek_min','CountWeekend_min','U-Stat_min','P-Val_min','weekendCount_min','weekCount_min',
                 'table_std','Column_std','CountWeek_std','CountWeekend_std','U-Stat_std','P-Val_std','weekendCount_std','weekCount_std']
            
        print("Total Amount of Each Item")
        print(table_label)
        data1 = pd.DataFrame(data1,columns=colss)
        
        for z in ['mean','max','min','std']:
            chunk = data1[['table_'+z,'Column_'+z,'CountWeek_'+z,'CountWeekend_'+z,'U-Stat_'+z,'P-Val_'+z,'weekendCount_'+z,'weekCount_'+z]]
            chunk['analysis'] = 'Total Amount of Each Item, no nans ({})'.format(z)
            chunk = chunk.rename(columns={'table_'+z:'table','Column_'+z:'column','CountWeek_'+z:'week','CountWeekend_'+z:'weekend','U-Stat_'+z:'U-Stat','P-Val_'+z:'P-Val',
                                         'weekendCount_'+z:'weekendCount','weekCount_'+z:'weekCount'})
            chunk = chunk[((chunk['U-Stat']>0)&(chunk['P-Val']>0))|((chunk['U-Stat']<0)&(chunk['P-Val']<0))]
            display(chunk.head())
            allResults = allResults.append(chunk)
        
        data2 = pd.DataFrame(data2,columns=colss)
        
        for z in ['mean','max','min','std']:
            chunk = data2[['table_'+z,'Column_'+z,'CountWeek_'+z,'CountWeekend_'+z,'U-Stat_'+z,'P-Val_'+z,'weekendCount_'+z,'weekCount_'+z]]
            chunk['analysis'] = 'Total Amount of Each Item, no 0s ({})'.format(z)
            chunk = chunk.rename(columns={'table_'+z:'table','Column_'+z:'column','CountWeek_'+z:'week','CountWeekend_'+z:'weekend','U-Stat_'+z:'U-Stat','P-Val_'+z:'P-Val',
                                         'weekendCount_'+z:'weekendCount','weekCount_'+z:'weekCount'})
            chunk = chunk[((chunk['U-Stat']>0)&(chunk['P-Val']>0))|((chunk['U-Stat']<0)&(chunk['P-Val']<0))]
            display(chunk.head())
            allResults = allResults.append(chunk)
        
        #data = data.dropna()
        #display(data[((data['U-Stat_mean']>0)&(data['P-Val_mean']>0))|((data['U-Stat_max']>0)&(data['P-Val_max']>0))|((data['U-Stat_min']>0)&(data['P-Val_min']>0))]) 
        print(' ')

MICROBIOLOGYEVENTS
LABEVENTS


/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:134: RuntimeWarning: Mean of empty slice
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:134: RuntimeWarning: All-NaN axis encountered
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type

Total Amount of Each Item
LABEVENTS


/afs/crc.nd.edu/user/l/lfaust/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis
23,LABEVENTS,Temperature,36.745080,37.082857,-0.675782,-0.050950,208.0,82.0,"Total Amount of Each Item, no nans (mean)"
39,LABEVENTS,"Potassium, Whole Blood",4.324306,4.813485,-0.924444,-0.017684,70.0,33.0,"Total Amount of Each Item, no nans (mean)"
55,LABEVENTS,"Creatine Kinase, MB Isoenzyme",16.567471,24.869231,-5.753755,-0.420382,145.0,65.0,"Total Amount of Each Item, no nans (mean)"
62,LABEVENTS,Lactate Dehydrogenase (LD),1098.349617,1393.153509,-136.833763,-23.650156,174.0,76.0,"Total Amount of Each Item, no nans (mean)"
69,LABEVENTS,Potassium,4.285988,4.450711,-0.355521,-0.006213,295.0,129.0,"Total Amount of Each Item, no nans (mean)"


...

,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


,table,column,week,weekend,U-Stat,P-Val,weekendCount,weekCount,analysis


 
CPTEVENTS
DATETIMEEVENTS
NOTEEVENTS
CHARTEVENTS


In [ ]:
allResults.to_csv('results.csv')

In [ ]:
caregiver_data = set(['DATETIMEEVENTS','PROCEDUREEVENTS_MV','NOTEEVENTS','INPUTITEMS'])
#Caregivers there is research that says it is due to a decrease in specialist care
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    if table_label in caregiver_data:
        data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)

        a = data_weekend.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values.tolist()
        b = data_week.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values.tolist()
        t = HDIofMCMC(BESTmcmc(a,b,numSavedSteps=10000))

        c = data_weekend.groupby('HADM_ID').size().values.tolist()
        d = data_week.groupby('HADM_ID').size().values.tolist()
        t1 = HDIofMCMC(BESTmcmc(c,d,numSavedSteps=10000))
        columns = ['Table','U-Stat-Unique','P-Value','Mean Weekend','Mean Week','U-Stat-Total','P-Value','Mean Weekend','Mean Week']
        data = pd.DataFrame([[table_label,t[0],t[1],np.mean(a),np.mean(b),t1[0],t1[1],np.mean(c),np.mean(d)]],columns = columns)
        display(data)   
        print(' ')

In [ ]:
#Nursing Notes Breakdown
nursenotes_data = set(['NOTEEVENTS'])
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    if table_label in nursenotes_data:
        data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)

        for category in [['ECG'],['Nursing/other','Nursing'],['Radiology'],['Echo'],['Discharge summary'],['Physician '],['Nutrition'],['General'],
               ['Case Management '],['Respiratory '],['Rehab Services'],['Social Work'],['Pharmacy'],['Consult']]:

            data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)

            data_weekend = data_weekend[data_weekend['CATEGORY'].isin(category)]
            data_week = data_week[data_week['CATEGORY'].isin(category)]

            a = data_weekend.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values
            b = data_week.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values
            try:
                t = HDIofMCMC(BESTmcmc(a,b,numSavedSteps=10000))
            except:
                t = (np.nan,np.nan)

            c = data_weekend.groupby('HADM_ID').size()
            d = data_week.groupby('HADM_ID').size()
            try:
                t1 = HDIofMCMC(BESTmcmc(c,d,numSavedSteps=10000))
            except:
                t1 = (np.nan,np.nan)

            columns = ['Category','U-Stat-Unique','P-Value','Mean Weekend','Mean Week','U-Stat-Total','P-Value','Mean Weekend','Mean Week']
            data = pd.DataFrame([[category,t[0],t[1],np.mean(a),np.mean(b),t1[0],t1[1],np.mean(c),np.mean(d)]],columns=columns)
            display(data)   

In [ ]:
#Durations
icu_stay_table = pd.read_csv('../MIMICIII/ICUSTAYS.csv')
icu_stay_table = icu_stay_table[icu_stay_table.SUBJECT_ID.isin(expired_pat_week.pat_id.append(expired_pat_weekend.pat_id).values)]
icu_stay_table.loc[icu_stay_table.SUBJECT_ID.isin(expired_pat_weekend.pat_id.values),'weekend'] = 1
icu_stay_table.fillna(0,inplace=True)
icu_stay_table.INTIME = pd.to_datetime(icu_stay_table.INTIME)
icu_stay_table['end'] = icu_stay_table.INTIME + timedelta(hours=time_span)
icu_stay_table.INTIME = pd.to_datetime(icu_stay_table.INTIME)
icu_stay_table = icu_stay_table[['ROW_ID','SUBJECT_ID','HADM_ID','ICUSTAY_ID','INTIME','end','weekend']]
icu_stay_table['duration'] = np.nan

for table in ['adenosinedurations','crrtdurations','dobutaminedurations','dopaminedurations','epinephrinedurations','isupreldurations','milrinonedurations','norepinephrinedurations',
              'phenylephrinedurations','vasopressindurations','vasopressordurations','ventdurations']:
              #'weightdurations']:
    print(table)
    stay = icu_stay_table.copy()
    durations = pd.read_csv('../views/{}.csv'.format(table))
    for index, row in durations.iterrows():
        patient = stay[stay.ICUSTAY_ID == row.icustay_id]
        if(not patient.empty):
            durationStart = pd.to_datetime(row.starttime)
            if(durationStart > pd.to_datetime(patient.INTIME.values[0]) and durationStart < pd.to_datetime(patient['end'].values[0])):             
                durationAmount = row.duration_hours
                #if duration end time exceeds 24 hour mark shorten duration
                if pd.to_datetime(row.endtime) > pd.to_datetime(patient['end'].values[0]):
                    durationAmount = pd.to_datetime(patient['end'].values[0]) - pd.to_datetime(row.starttime)
                    durationAmount = durationAmount.seconds / 3600
                if(patient.duration.isnull().values[0] == True):
                    stay.at[patient.index,'duration'] = durationAmount
                else:
                    stay.at[patient.index,'duration'] = durationAmount + stay[stay.ICUSTAY_ID == row.icustay_id].duration.values[0]
    stay = stay[stay.duration.notnull()]
    print('week: {}'.format(stay[stay.weekend == 0].duration.mean()))
    print('weekend: {}'.format(stay[stay.weekend == 1].duration.mean()))
    a = stay[stay.weekend == 1].duration.values.tolist()
    b = stay[stay.weekend == 0].duration.values.tolist()
    if(len(a)>0 and len(b)>0):
        print(HDIofMCMC(BESTmcmc(a,b,numSavedSteps=10000)))
    print()